In [1]:
import os
import pandas as pd
import sqlite3

In [9]:
files={}
path='data/'
for dir_name,_,filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.csv'):
            files[filename[:-4]]=os.path.join(dir_name, filename)
            print(os.path.join(dir_name, filename))

data/customer.csv
data/cust_account.csv
data/cust_churn.csv
data/cust_loc.csv
data/cust_services.csv
data/sample.csv


In [13]:
def table_info(conn,cur):
    '''
    conn: Database connection object
    cur: Database cursor object
    
    Lists tables present in the database and columns associated with each table
    '''
    tables=cur.execute('SELECT name from sqlite_master where type=\'table\';').fetchall()
    
    for x in tables:
        table_name=x[0]
        table=pd.read_sql('SELECT * FROM {} LIMIT 0'.format(table_name),conn)
        print(table_name)
        
        for col in table.columns:
            print('\t'+col)
        print()
    return None

In [14]:

conn=sqlite3.connect('data/churn.db')
cur=conn.cursor()

for filename in files:
    file=pd.read_csv(files[filename])
    file.to_sql(filename,conn,if_exists='replace',index=False)
    
conn.commit()
table_info(conn,cur)
conn.close()


customer
	CustomerID
	Gender
	SeniorCitizen
	Partner
	Dependents

cust_account
	Account_id
	Tenure
	Contract
	PaymentMethod
	PaperlessBilling
	MonthlyCharges
	TotalCharges

cust_churn
	Id
	Churn

cust_loc
	Cust_ID
	State
	Latitude
	Longitude
	ZipCode

cust_services
	Cust_ID
	PhoneService
	MultipleLines
	InternetService
	OnlineSecurity
	OnlineBackup
	DeviceProtection
	TechSupport
	StreamingTV
	StreamingMovies

sample
	Name
	Age

